In [1]:
import cv2
import os
import time
from datetime import datetime

# Path to save the recorded video
save_dir = r"C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset"
os.makedirs(save_dir, exist_ok=True)

# Initialize the webcam
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Warm up the camera
print("Warming up the camera...")
for _ in range(10):
    ret, _ = cap.read()
    cv2.waitKey(100)  # 100ms delay

print("Camera warmed up. Press 's' to start recording for 10 seconds.")

recording = False
start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    cv2.imshow("Live Feed - Press 's' to start recording", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('s') and not recording:
        # Start recording
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"recording_{timestamp}.mp4"
        output_path = os.path.join(save_dir, filename)

        # Define video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = 20.0
        height, width, _ = frame.shape
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        print(f"Recording started: {output_path}")
        recording = True
        start_time = time.time()

    if recording:
        out.write(frame)
        if time.time() - start_time > 5:  # 10 seconds
            print("Recording finished.")
            recording = False
            out.release()

    if key == ord('q'):
        print("Exiting.")
        break

cap.release()
cv2.destroyAllWindows()


Warming up the camera...
Camera warmed up. Press 's' to start recording for 10 seconds.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_034839.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_034903.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_034922.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_035001.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_035146.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_20250717_035156.mp4
Recording finished.
Recording started: C:\work\personal\Projects\Noodle Pack Object Detection\raw_dataset\recording_

In [2]:
import cv2
from pupil_apriltags import Detector

cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

detector = Detector(
    families='tag36h11',
    nthreads=4,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=True,
    decode_sharpening=0.25,
    debug=False
)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    tags = detector.detect(gray)

    for tag in tags:
        corners = [(int(x), int(y)) for x, y in tag.corners]
        for i in range(4):
            pt1 = corners[i]
            pt2 = corners[(i + 1) % 4]
            cv2.line(frame, pt1, pt2, (0, 255, 0), 2)

        # Draw center
        cX, cY = int(tag.center[0]), int(tag.center[1])
        cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)

        # Display Tag ID
        cv2.putText(frame, f"ID: {tag.tag_id}", (cX - 20, cY - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    cv2.imshow("AprilTag Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
